In [1]:
%matplotlib widget

In [2]:
import numpy as np
import ipyvolume as ipv
from scipy.spatial import Delaunay
from scipy.spatial.transform import Rotation as rot

# plotting
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib.pyplot as plt

#interactive plots
import ipywidgets as widgets
from ipywidgets import VBox, HBox, IntSlider, Checkbox, interactive_output, FloatSlider
from IPython.display import display

import libo
from libo.io import tc3

import weldx.geometry as geo
import weldx.transformations as tf
import weldx.utility as ut
import weldx.visualization as vs


Import movement data

In [3]:
data = tc3.read_db(17519)
ds = tc3.to_xarray(data)
ds

<xarray.Dataset>
Dimensions:         (time: 102261)
Coordinates:
  * time            (time) datetime64[ns] 2020-02-19T15:35:15.151000 ... 2020-02-19T15:36:57.543000
Data variables:
    trigScan2_fire  (time) float64 nan nan nan nan 0.0 ... 0.0 0.0 0.0 0.0 nan
    trigLLT2        (time) float64 nan nan nan nan 0.0 ... 0.0 0.0 0.0 0.0 nan
    trigScan2_prog  (time) float64 nan nan nan nan 0.0 ... 0.0 0.0 0.0 0.0 nan
    trigXiris1      (time) float64 nan nan nan nan 0.0 ... 0.0 0.0 0.0 0.0 nan
    trigHDR1_prog   (time) float64 nan nan nan nan 0.0 ... 0.0 0.0 0.0 0.0 nan
    trigLLT1        (time) float64 nan nan nan nan 0.0 ... 0.0 0.0 0.0 0.0 nan
    trigScan1_fire  (time) float64 nan nan nan nan 0.0 ... 0.0 0.0 0.0 0.0 nan
    trigScan1_prog  (time) float64 nan nan nan nan 0.0 ... 0.0 0.0 0.0 0.0 nan
    trigProgramm    (time) float64 nan nan nan nan 0.0 ... 0.0 0.0 0.0 0.0 nan
    trigSchweissen  (time) float64 nan nan nan nan 0.0 ... 0.0 0.0 0.0 0.0 nan
    JobBetrieb      (time) float64 nan nan nan nan 0.0 ... 1.0 1.0 1.0 1.0 nan
    LiBoSteht       (time) float64 nan nan nan nan 0.0 ... 0.0 0.0 0.0 0.0 nan
    S1_ax1          (time) float64 nan nan nan nan 68.0 ... 68.0 68.0 68.0 nan
    S1_ax2          (time) float64 nan nan nan nan 60.0 ... 60.0 60.0 60.0 nan
    schw_NR         (time) float64 nan nan nan nan 1.0 ... 1.0 1.0 1.0 1.0 nan
    UPDATE_TRIGGER  (time) float64 nan nan nan nan 1.0 ... 0.0 0.0 0.0 0.0 nan
    FB_Ry           (time) float64 nan 0.1615 nan nan nan ... nan nan nan nan
    VTCP_V3         (time) float64 nan 14.06 nan nan nan ... nan nan nan nan nan
    speed_override  (time) float64 nan 0.0 nan nan nan ... nan nan nan nan nan
    V               (time) float64 nan 13.59 nan nan nan ... nan nan nan nan nan
    FB_Rz           (time) float64 nan -0.4152 nan nan nan ... nan nan nan nan
    FB_Rx           (time) float64 nan -179.9 nan nan nan ... nan nan nan nan
    X               (time) float64 nan 47.23 nan nan nan ... nan nan nan nan nan
    FB_Y            (time) float64 nan 1.045e+03 nan nan nan ... nan nan nan nan
    FB_X            (time) float64 nan 47.18 nan nan nan ... nan nan nan nan nan
    Rz              (time) float64 nan -0.4144 nan nan nan ... nan nan nan nan
    Ry              (time) float64 nan 0.1624 nan nan nan ... nan nan nan nan
    Rx              (time) float64 nan -179.9 nan nan nan ... nan nan nan nan
    Z               (time) float64 nan -137.7 nan nan nan ... nan nan nan nan
    Y               (time) float64 nan 1.045e+03 nan nan nan ... nan nan nan nan
    FB_Z            (time) float64 nan -137.3 nan nan nan ... nan nan nan nan
    volumeFlow      (time) float64 nan nan 0.26 nan nan ... nan nan nan 0.27 nan
    werkstueck_ID   (time) float64 364.0 nan nan nan nan ... nan nan nan nan nan
    naht_NR         (time) float64 nan nan nan 1.0 nan ... nan nan nan nan 1.0
    lage_NR         (time) float64 nan nan nan 1.0 nan ... nan nan nan nan 1.0
    raupe_NR        (time) float64 nan nan nan 1.0 nan ... nan nan nan nan 1.0
Attributes:
    scopeName:         TC3 Export Tool
    svdFile:           D:\Messdaten\2020_02_19_16_36_57_MH24_MAIN_AutoSave.txt
    StartTimeTC3:      132266001152460000
    EndTimeTC3:        132266002175010000
    RunTime:           102.255
    StartTime:         2020-02-19 16:35:15.246000 +0100
    EndTime:           2020-02-19 16:36:57.501000 +0100
    hostname:          WS7096
    version:           0.2.2
    TC3Import:         Python3
    filename:          
    filepath:          
    RunTime_TimeUnit:  s
    file_id:           17519

In [33]:
ds[["FB_X","FB_Y","FB_Z"]]

# schweißen
plt.figure()
ds.trigScan2_fire.dropna("time").plot()
ds.trigProgramm.dropna("time").plot()
ds.trigSchweissen.dropna("time").plot()
#ds.trigScan2_prog.dropna("time").plot()
ds.JobBetrieb.dropna("time").plot()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.figure()
ds.FB_Rz.dropna("time").diff("time").plot()

In [44]:
np.argmax(ds.FB_Rz.dropna("time").diff("time").data)
ds.trigSchweissen

<xarray.DataArray 'trigSchweissen' (time: 102261)>
array([nan, nan, nan, ...,  0.,  0., nan])
Coordinates:
  * time     (time) datetime64[ns] 2020-02-19T15:35:15.151000 ... 2020-02-19T15:36:57.543000
Attributes:
    SymbolComment:     
    DataType:          UINT8
    SampleTime:        1.0
    VariableSize:      1
    SymbolBased:       1
    IndexGroup:        16448
    IndexOffset:       513978
    SymbolName:        MAIN.fbVersuchsstand.fbDX200.trigSchweissen
    NetID:             169.254.180.209.1.1
    Port:              851
    Offset:            0.0
    ScaleFactor:       1.0
    BitMask:           0xffffffffffffffff
    Unit:              (None)
    Unit_ScaleFactor:  1
    Unit_Offset:       0
    TimeUnit:          ms
    StartTimeUTC:      2020-02-19 15:35:15.246000
    StartTimeUTC_fmt:  %Y-%m-%d %H:%M:%S.%f
    count:             102256.0
    mean:              0.5771495071193866
    std:               0.49401451417129616
    min:               0.0
    max:               1.0
    p25:               0.0
    p50:               1.0
    p75:               1.0

In [30]:
locations = ds[["FB_X","FB_Y","FB_Z"]].where(ds.trigSchweissen==1).dropna("time").to_array().data
#locations = np.array([ds.FB_X.dropna("time").data, ds.FB_Y.dropna("time").data, ds.FB_Z.dropna("time").data], float)
locations

array([[  47.188,   47.195,   47.206, ...,   49.738,   49.738,   49.738],
       [1045.312, 1045.315, 1045.31 , ..., 1375.272, 1375.272, 1375.272],
       [-137.386, -137.452, -137.512, ..., -136.894, -136.894, -136.894]])

In [6]:
rotations = ds[["FB_Rx","FB_Ry","FB_Rz"]].where(ds.trigSchweissen==1).dropna("time").to_array().data
rotations

array([[-1.798628e+02, -1.798628e+02, -1.798633e+02, ..., -1.798663e+02,
        -1.798663e+02, -1.798663e+02],
       [ 1.615000e-01,  1.623000e-01,  1.623000e-01, ...,  1.463000e-01,
         1.463000e-01,  1.463000e-01],
       [-4.146000e-01, -4.154000e-01, -4.155000e-01, ..., -4.037000e-01,
        -4.037000e-01, -4.037000e-01]])

In [7]:
#plt.plot(locations[0],locations[1])
fig = plt.figure()
ax = fig.gca(projection='3d')
fig.canvas.layout.height="500px"
fig.canvas.layout.width="500px"
ax.plot(locations[0], locations[1], locations[2])
#vs.set_axes_equal(ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Define coordinate systems:
- `cs_base` is the root coordinate system
- `cs_ref` is defined in `cs_base` -> coordinate system created from 3 robot positions
- `cs_sp` is defined in `cs_ref` -> specimen coordinate system with x pointing into the direction of the trace

Important note: Measured data is not perfectly orthogonal. We use the cross product to get a z axis which is perfectly orthogonal (in floating point precision) and recalculate a perfectly orthogonal y-axis. There are certainly better ways to adress this problem.

In [8]:
origin = ut.to_float_array([67.311, 1055.137, -132.694])
offset_ox = ut.to_float_array([157.345, 1054.486,-132.949 ])
offset_oy = ut.to_float_array([69.149, 1366.014, -131.962]) 

vec_x = offset_ox - origin
vec_y = offset_oy - origin
vec_z = np.cross(vec_x, vec_y)

cs_ref = tf.LocalCoordinateSystem.construct_from_xz_and_orientation(vec_x=vec_x, vec_z=vec_z, origin=origin)
cs_sp = tf.LocalCoordinateSystem(basis=tf.rotation_matrix_z(np.pi/2),origin=[-20, -10 ,-8])

cs_sp_in_base = cs_sp + cs_ref

Transform movement data to specimen coordinate system:

In [9]:
locations_sp = np.matmul(cs_sp_in_base.orientation.transpose(),locations- cs_sp_in_base.location[:,np.newaxis]) 

fig2 = plt.figure()
ax2 = fig2.gca(projection='3d')
fig2.canvas.layout.height="500px"
fig2.canvas.layout.width="500px"
ax2.plot(locations_sp[0], locations_sp[1], locations_sp[2])
ax2.set_xlabel("x")
ax2.set_ylabel("y")
ax2.set_zlabel("z")
vs.set_axes_equal(ax2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# create points
p_0 = [40, 8]
p_1_1 = [np.tan(3/18 *np.pi)*8, 8]
p_1_2 = [np.tan(2/18 *np.pi)*8, 8]
p_2 = [0, 0]
p_3 = [40, 0]

# create shapes
s_p1_r = geo.Shape().add_line_segments([p_0, p_1_1, p_2])
s_p2_r = geo.Shape().add_line_segments([p_0, p_1_2, p_2])
s_p1_l = s_p1_r.reflect([1, 0])
s_p2_l = s_p2_r.reflect([1, 0])

# create profiles
p_1 = geo.Profile([s_p1_l, s_p1_r])
p_2 = geo.Profile([s_p2_l, s_p2_r])

# rasterize and plot profiles
data_p_1 = p_1.rasterize(1)
data_p_2 = p_2.rasterize(1)

fig3 = plt.figure()
plt.plot(data_p_1[0], data_p_1[1], "ro", label="profile 1")
plt.plot(data_p_2[0], data_p_2[1], "bo", label="profile 2")
plt.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Create variable profile, trace and geometry

In [11]:
vp = geo.VariableProfile([p_1, p_2], [0, 1],[geo.linear_profile_interpolation_sbs])
trace = geo.Trace(geo.LinearHorizontalTraceSegment(350))

g_0 = geo.Geometry(vp,trace)

In [12]:
data_g_0 = g_0.rasterize(profile_raster_width=4, trace_raster_width=4)

Plot everything together

In [13]:
fig4 = plt.figure()
ax4 = fig4.gca(projection='3d')
fig4.canvas.layout.height="1500px"
fig4.canvas.layout.width="1500px"
ax4.plot(locations_sp[0], locations_sp[1], locations_sp[2], "b")
ax4.plot(data_g_0[0], data_g_0[1], data_g_0[2], "xr")
ax4.set_xlabel("x")
ax4.set_ylabel("y")
ax4.set_zlabel("z")
#vs.set_axes_equal(ax4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'z')

In [14]:
scan_L0 = libo.io.tools.merge_scan_tcp(scan=17507,tcp=17518)

In [15]:
scan_L0.scan_tcp

<xarray.DataArray 'scan_tcp' (profile: 1598, tcp_variable: 6)>
array([[ 4.70801250e+01,  1.02531125e+03, -1.17707875e+02,
        -1.79865300e+02,  1.62400000e-01,  8.95856000e+01],
       [ 4.70817500e+01,  1.02550100e+03, -1.17707000e+02,
        -1.79865300e+02,  1.62400000e-01,  8.95856000e+01],
       [ 4.70836250e+01,  1.02573100e+03, -1.17707000e+02,
        -1.79865300e+02,  1.62400000e-01,  8.95856000e+01],
       ...,
       [ 4.97393750e+01,  1.39257000e+03, -1.16844000e+02,
        -1.79865300e+02,  1.62400000e-01,  8.95856000e+01],
       [ 4.97406250e+01,  1.39280000e+03, -1.16844000e+02,
        -1.79865300e+02,  1.62400000e-01,  8.95856000e+01],
       [ 4.97427500e+01,  1.39303000e+03, -1.16843125e+02,
        -1.79865300e+02,  1.62400000e-01,  8.95856000e+01]])
Coordinates:
  * profile       (profile) uint64 1 2 3 4 5 6 ... 1595 1596 1597 1598 1599 1600
    time          (profile) datetime64[ns] 2020-02-19T15:31:20.263000 ... 2020-02-19T15:31:36.253000
  * tcp_variable  (tcp_variable) <U2 'X' 'Y' 'Z' 'Rx' 'Ry' 'Rz'
Attributes:
    scopeName:         TC3 Export Tool
    svdFile:           D:\Messdaten\2020_02_19_16_31_38_MH24_MAIN_AutoSave.txt
    StartTimeTC3:      132265997981750000
    EndTimeTC3:        132265998980700000
    RunTime:           99.895
    StartTime:         2020-02-19 16:29:58.175000 +0100
    EndTime:           2020-02-19 16:31:38.070000 +0100
    hostname:          WS7096
    version:           0.2.2
    TC3Import:         Python3
    filename:          
    filepath:          
    RunTime_TimeUnit:  s
    file_id:           17518

In [16]:
angles = scan_L0.scan_tcp.sel(tcp_variable=["Rx","Ry","Rz"]).data

In [17]:
rotation_matrices = rot.from_euler(angles=angles,seq="xyz",degrees=True).as_dcm()

In [18]:
locations = scan_L0.scan_tcp.sel(tcp_variable=["X","Y","Z"]).data

In [19]:
idx = 1300

cs_scan_0 = tf.LocalCoordinateSystem(basis=rotation_matrices[idx], origin = locations[idx])
cs_scan_0_in_sp = cs_scan_0 - cs_ref - cs_sp
cs_scan_0_in_sp.origin

array([2.79483357e+02, 5.58452886e-02, 2.29997900e+01])

In [20]:
scan_L0.sel(profile=600).scan_tcp

<xarray.DataArray 'scan_tcp' (tcp_variable: 6)>
array([ 4.807775e+01,  1.163037e+03, -1.173840e+02, -1.798653e+02,
        1.624000e-01,  8.958560e+01])
Coordinates:
    profile       uint64 600
    time          datetime64[ns] 2020-02-19T15:31:26.253000
  * tcp_variable  (tcp_variable) <U2 'X' 'Y' 'Z' 'Rx' 'Ry' 'Rz'
Attributes:
    scopeName:         TC3 Export Tool
    svdFile:           D:\Messdaten\2020_02_19_16_31_38_MH24_MAIN_AutoSave.txt
    StartTimeTC3:      132265997981750000
    EndTimeTC3:        132265998980700000
    RunTime:           99.895
    StartTime:         2020-02-19 16:29:58.175000 +0100
    EndTime:           2020-02-19 16:31:38.070000 +0100
    hostname:          WS7096
    version:           0.2.2
    TC3Import:         Python3
    filename:          
    filepath:          
    RunTime_TimeUnit:  s
    file_id:           17518

In [21]:
scan_data = scan_L0.scan_line.sel(profile = idx + 1).data.transpose()

fig = plt.figure()
plt.plot(scan_data[1],scan_data[2])
plt.xlim([-10,10])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-10, 10)

In [22]:
cs_profile = tf.LocalCoordinateSystem(basis = [[1,0,0],[0,1,0],[0,0,-1]],origin=[0,0,260])
cs_profile_in_sp = cs_profile + cs_scan_0_in_sp
cs_profile_in_sp.orientation

array([[ 9.99986557e-01, -7.70089743e-06,  5.18522090e-03],
       [-1.01961915e-05, -9.99999884e-01,  4.81205715e-04],
       [-5.18521659e-03,  4.81252116e-04,  9.99986441e-01]])

In [23]:
#scan_data_sp = np.matmul(cs_profile_in_sp.orientation.transpose(),scan_data - cs_profile_in_sp.location[:,np.newaxis])
scan_data_sp = np.matmul(cs_profile_in_sp.orientation,scan_data) + cs_profile_in_sp.location[:,np.newaxis]

In [24]:
fig6 = plt.figure()
ax6 = fig6.gca(projection='3d')
fig6.canvas.layout.height="1000px"
fig6.canvas.layout.width="1000px"
ax6.plot(locations_sp[0], locations_sp[1], locations_sp[2], "oy")
#ax6.plot(data_g_0[0], data_g_0[1], data_g_0[2], ",k")
ax6.plot_trisurf(data_g_0[0], data_g_0[1], data_g_0[2], color="y", zsort=True)
#vs.plot_coordinate_system(cs_profile_in_sp,ax6)
ax6.set_xlabel("x")
ax6.set_ylabel("y")
ax6.set_zlabel("z")
ax6.plot(scan_data_sp[0], scan_data_sp[1], scan_data_sp[2], "r")
#vs.set_axes_equal(ax6)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/opt/tljh/user/envs/libo/lib/python3.7/site-packages/ipykernel_launcher.py:7: MatplotlibDeprecationWarning: Passing True to mean 'average' for set_zsort is deprecated and support will be removed in Matplotlib 3.3; pass 'average' instead.
  import sys


In [25]:
tri = Delaunay(data_g_0.transpose()[:,:2])
triangles = tri.simplices

In [26]:
triangle_points = data_g_0.transpose()[triangles]
x = data_g_0[0]
y = data_g_0[1]
z = data_g_0[2]
triangle_points

array([[[  0.        ,  36.06875579,   8.        ],
        [  3.97727273,  40.        ,   8.        ],
        [  0.        ,  40.        ,   8.        ]],

       [[  3.97727273,  24.26640176,   8.        ],
        [  0.        ,  20.34377897,   8.        ],
        [  3.97727273,  20.3330022 ,   8.        ]],

       [[290.34090909, -11.37990473,   8.        ],
        [286.36363636,  -7.30856252,   8.        ],
        [286.36363636, -11.3949922 ,   8.        ]],

       ...,

       [[214.77272727,  15.71420011,   8.        ],
        [218.75      ,  15.70126799,   8.        ],
        [218.75      ,  19.75105665,   8.        ]],

       [[222.72727273,  19.74027989,   8.        ],
        [218.75      ,  15.70126799,   8.        ],
        [222.72727273,  15.68833586,   8.        ]],

       [[218.75      ,  15.70126799,   8.        ],
        [222.72727273,  19.74027989,   8.        ],
        [218.75      ,  19.75105665,   8.        ]]])

In [27]:
ipv.figure()
# we draw the tetrahedron
mesh = ipv.plot_trisurf(x, y, z, triangles=triangles, color= [0.6,0.6,0.6])
ipv.scatter(locations_sp[0], locations_sp[1], locations_sp[2], size=1, marker="sphere", color=[0.3,0.9,0.7])
ipv.scatter(scan_data_sp[0], scan_data_sp[1], scan_data_sp[2], size=1, marker="sphere", color='red')
ipv.xlim(0,350)
ipv.ylim(-30,30)
ipv.zlim(-10,30)
ipv.show()

In [28]:
import ipyvolume as ipv
x, y, z, u, v = ipv.examples.klein_bottle(draw=False)
ipv.figure()
m = ipv.plot_mesh(x, y, z, wireframe=False)
ipv.squarelim()
ipv.show()